# Introduction to Data Science 2025

# Week 2

## Exercise 1 | Titanic: data preprocessing and imputation
<span style="font-weight: bold"> *Note: You can find tutorials for NumPy and Pandas under 'Useful tutorials' in the course material.*</span>

Download the [Titanic dataset](https://www.kaggle.com/c/titanic) [train.csv] from Kaggle or <span style="font-weight: 500">directly from the course material</span>, and complete the following exercises. If you choose to download the dataset from Kaggle, you will need to create a Kaggle account unless you already have one, but it is quite straightforward.

The dataset consists of personal information of all the passengers on board the RMS Titanic, along with information about whether they survived the iceberg collision or not.

1. Your first task is to read the data file and print the shape of the data.

    <span style="font-weight: 500"> *Hint 1: You can read them into a Pandas dataframe if you wish.*</span>
    
    <span style="font-weight: 500"> *Hint 2: The shape of the data should be (891, 12).*</span>

In [ ]:
# Use this cell for your code
import pandas as pd

file_path = "train.csv"
df = pd.read_csv(file_path)

print("Shape:", df.shape)

2. Let's look at the data and get started with some preprocessing. Some of the columns, e.g <span style="font-weight: 500"> *Name*</span>, simply identify a person and are not useful for prediction tasks. Try to identify these columns, and remove them.

    <span style="font-weight: 500"> *Hint: The shape of the data should now be (891, 9).*</span>

In [ ]:
# Use this cell for your code
df = df.drop(columns=['Name', 'Sex', 'Age'])

3. The column <span style="font-weight: 500">*Cabin*</span> contains a letter and a number. A smart catch at this point would be to notice that the letter stands for the deck level on the ship. Keeping just the deck information would be more informative when developing, e.g. a classifier that predicts whether a passenger survived. The next step in our preprocessing will be to add a new column to the dataset, which consists simply of the deck letter. You can then remove the original <span style="font-weight: 500">*Cabin*</span>-column.

<span style="font-weight: 500">*Hint: The deck letters should be ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'T'].*</span>

In [ ]:
# Use this cell for your code
df["Level"] = df["Cabin"].str[0]
df = df.drop(columns=['Name', 'Sex', 'Age','Cabin'])

4. You’ll notice that some of the columns, such as the previously added deck number, are [categorical](https://en.wikipedia.org/wiki/Categorical_variable). To preprocess the categorical variables so that they're ready for further computation, we need to avoid the current string format of the values. This means the next step for each categorical variable is to transform the string values to numeric ones, that correspond to a unique integer ID representative of each distinct category. This process is called label encoding and you can read more about it [here](https://pandas.pydata.org/docs/user_guide/categorical.html).

    <span style="font-weight: 500">*Hint: Pandas can do this for you.*</span>

In [ ]:
# Use this cell for your code
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].astype("category").cat.codes

5. Next, let's look into missing value **imputation**. Some of the rows in the data have missing values, e.g when the cabin number of a person is unknown. Most machine learning algorithms have trouble with missing values, and they need to be handled during preprocessing:

    a) For continuous variables, replace the missing values with the mean of the non-missing values of that column.

    b) For categorical variables, replace the missing values with the mode of the column.

    <span style="font-weight: 500">*Remember: Even though in the previous step we transformed categorical variables into their numeric representation, they are still categorical.*</span>

In [ ]:
# Use this cell for your code
for col in df.columns:
    if df[col].dtype in ["float64", "int64"]:
          mean_value = df[col].mean()
          df[col] = df[col].fillna(mean_value)
    else: 
        mode_value = df[col].mode()[0]
        df[col] = df[col].fillna(mode_value)

6. At this point, all data is numeric. Write the data, with the modifications we made, to a  <span style="font-weight: 500"> .csv</span> file. Then, write another file, this time in <span style="font-weight: 500">JSON</span> format, with the following structure:

In [6]:
#[
#    {
#        "Deck": 0,
#        "Age": 20,
#        "Survived", 0
#        ...
#    },
#    {
#        ...
#    }
#]

In [ ]:
# Use this cell for your code
csv_output = "output.csv"
df.to_csv(csv_output, index=False)
json_output = "output.json"
df.to_json(json_output, orient="records", indent=4)

Study the records and try to see if there is any evident pattern in terms of chances of survival.

**Remember to submit your code on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**

## Exercise 2 | Titanic 2.0: exploratory data analysis

In this exercise, we’ll continue to study the Titanic dataset from the last exercise. Now that we have done some preprocessing, it’s time to look at the data with some exploratory data analysis.

1. First investigate each feature variable in turn. For each categorical variable, find out the mode, i.e., the most frequent value. For numerical variables, calculate the median value.

In [ ]:
# Use this cell for your code
for col in df.columns:
    if df[col].dtype == "float64" or df[col].dtype == "int64":
        median_value = df[col].median()
        print("Median:", median_value)
    else:
        mode_value = df[col].mode()[0]
        print("Mode:", mode_value)

2. Next, combine the modes of the categorical variables, and the medians of the numerical variables, to construct an imaginary “average survivor”. This "average survivor" should represent the typical passenger of the class of passengers who survived. Also following the same principle, construct the “average non-survivor”.

    <span style="font-weight: 500">*Hint 1: What are the average/most frequent variable values for a non-survivor?*</span>
    
    <span style="font-weight: 500">*Hint 2: You can split the dataframe in two: one subset containing all the survivors and one consisting of all the non-survivor instances. Then, you can use the summary statistics of each of these dataframe to create a prototype "average survivor" and "average non-survivor", respectively.*</span>

In [ ]:
# Use this cell for your code
sur = df[df["Survived"] == 1]
non_sur = df[df["Survived"] == 0]

def average_passenger(subset):
    avg_passenger = {}
    for col in subset.columns:
        if col == "Survived":
            avg_passenger[col] = subset[col].mode()[0]
        elif col in ["Embarked", "Level"]:
            avg_passenger[col] = subset[col].mode()[0]
        else: 
            avg_passenger[col] = subset[col].median()
    return avg_passenger

average_sur = average_passenger(sur)
average_non_sur = average_passenger(non_sur)

print("Average survivor:", average_sur)
print("Average non survivor:", average_non_sur)

3. Next, let's study the distributions of the variables in the two groups (survivor/non-survivor). How well do the average cases represent the respective groups? Can you find actual passengers that are very similar to the (average) representative of their own group? Can you find passengers that are very similar to the (average) representative of the other group?

    <span style="font-weight: 500">*Note: Feel free to choose EDA methods according to your preference: non-graphical/graphical, static/interactive - anything goes.*</span>

In [10]:
# Use this cell for your code

4. Next, let's continue the analysis by looking into pairwise and multivariate relationships between the variables in the two groups. Try to visualize two variables at a time using, e.g., scatter plots and use a different color to encode the survival status.

    <span style="font-weight: 500">*Hint 1: You can also check out Seaborn's pairplot function, if you wish.*</span>

    <span style="font-weight: 500">*Hint 2: To better show many data points with the same value for a given variable, you can use either transparency or ‘jitter’.*</span>

In [11]:
# Use this cell for your code

5. Finally, recall the preprocessing we did in the first exercise. What can you say about the effect of the choices that were made to use the mode and mean to impute missing values, instead of, for example, ignoring passengers with missing data?

*Use this (markdown) cell for your written answer*

**Remember to submit your code on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**

## Exercise 3 | Working with text data 2.0

This exercise is related to the second exercise from last week. Find the saved <span style="font-weight: 500">pos.txt</span> and <span style="font-weight: 500">neg.txt</span> files, or, alternatively, you can find the week 1 example solutions on the MOOC platform after Tuesday.

1. Find the most common words in each file (positive and negative). Examine the results. Do they tend to be general terms relating to the nature of the data? How well do they indicate positive/negative sentiment?

In [ ]:
# Use this cell for your code
pos_path = "pos.txt"
neg_path = "neg.txt"

with open(pos_path, "r", encoding="utf-8") as f:
    words_pos = f.read().split()
    
with open(neg_path, "r", encoding="utf-8") as f:
    words_neg = f.read().split()
    
def word_counter(words):
    freq = {}
    for w in words:
        freq[w] = freq.get(w,0) + 1
    return freq

freq_pos = word_counter(words_pos)
freq_neg = word_counter(words_neg)

common_words = set(freq_pos.keys()) & set(freq_neg.keys())

def most_common(freq_dict):
    most_common_word = None
    max_count = 0
    
    for w, count in freq_dict.items():
        if count > max_count:
            most_common_word = w
            max_count = count
    return most_common_word, max_count

word_pos, count_pos = most_common(freq_pos)
word_neg, count_neg = most_common(freq_neg)

print("Most common word in pos.txt:", word_pos,",and appears:", count_pos,"times.")
print("Most common word in neg.txt:", word_neg,",and appears:", count_neg,"times.")

The most common word in the positive review document is 'great' and in the negative review document it is 'work'. 
You could state that great is positive and the work could refer to a more negative review, but you can't really
draw a conclusion from this.

2. Compute a [TF/IDF](https://en.wikipedia.org/wiki/Tf–idf) vector for each of the two text files, and make them into a <span style="font-weight: 500">2 x m</span> matrix, where <span style="font-weight: 500">m</span> is the number of unique words in the data. The problem with using the most common words in a review to analyze its contents is that words that are common overall will be common in all reviews (both positive and negative). This means that they probably are not good indicators about the sentiment of a specific review. TF/IDF stands for Term Frequency / Inverse Document Frequency (here the reviews are the documents), and is designed to help by taking into consideration not just the number of times a term occurs (term frequency), but also how many times a word exists in other reviews as well (inverse document frequency). You can use any variant of the formula, as well as off-the-shelf implementations. <span style="font-weight: 500">*Hint: You can use [sklearn](http://scikit-learn.org/).*</span>

In [ ]:
# Use this cell for your code
from sklearn.feature_extraction.text import TfidfVectorizer

pos_path = "pos.txt"
neg_path = "neg.txt"

with open(pos_path, "r", encoding="utf-8") as f:
    text_pos = f.read()
    
with open(neg_path, "r", encoding="utf-8") as f:
    text_neg = f.read()
    
documents = [text_pos, text_neg]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)
tfidf_matrix_dense = tfidf_matrix.toarray()
words = vectorizer.get_feature_names_out()

print("Shape:", tfidf_matrix_dense.shape)
print("Words:", words)
print(tfidf_matrix_dense)

3. List the words with the highest TF/IDF score in each class (positive | negative), and compare them to the most common words. What do you notice? Did TF/IDF work as expected?

In [ ]:
# Use this cell for your codefrom sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

pos_path = "pos.txt"
neg_path = "neg.txt"

with open(pos_path, "r", encoding="utf-8") as f:
    text_pos = f.read()
    
with open(neg_path, "r", encoding="utf-8") as f:
    text_neg = f.read()
    
documents = [text_pos, text_neg]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)
words = vectorizer.get_feature_names_out()
tfidf_dense = tfidf_matrix.toarray()

df_tfidf = pd.DataFrame(tfidf_dense, index=["pos.txt", "neg.txt"], columns=words)

def top_tfidf_words(row, n=10):
    sorted_idx = np.argsort(-row.values)[:n]
    return [(row.index[i], row.values[i]) for i in sorted_idx]

top_pos = top_tfidf_words(df_tfidf.loc["pos.txt"], n=10)
top_neg = top_tfidf_words(df_tfidf.loc["neg.txt"], n=10)

print("Top positive words:")
for word, score in top_pos:
    print(word,":", score)
    
print("Top negative words:")
for word, score in top_neg:
    print(word,":", score)

You get the same results as with the previous question, so that is no surprise. With this output I managed to print
a list of the most common words and you can see that the second most common word in pos.txt is work, which is the
most common word in neg.txt. 


4. Plot the words in each class with their corresponding TF/IDF scores. Note that there will be a lot of words, so you’ll have to think carefully to make your chart clear! If you can’t plot them all, plot a subset – think about how you should choose this subset.

    <span style="font-weight: 500">*Hint: you can use word clouds. But feel free to challenge yourselves to think of any other meaningful way to visualize this information!*</span>

In [ ]:
# Use this cell for your code
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pos_path = "pos.txt"
neg_path = "neg.txt"

with open(pos_path, "r", encoding="utf-8") as f:
    text_pos = f.read()
    
with open(neg_path, "r", encoding="utf-8") as f:
    text_neg = f.read()
    
documents = [text_pos, text_neg]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)
words = vectorizer.get_feature_names_out()
tfidf_dense = tfidf_matrix.toarray()

df_tfidf = pd.DataFrame(tfidf_dense, index=["pos.txt", "neg.txt"], columns=words)

def top_tfidf_words(row, n=20):
    sorted_idx = np.argsort(-row.values)[:n]
    return [(row.index[i], row.values[i]) for i in sorted_idx]

top_pos = top_tfidf_words(df_tfidf.loc["pos.txt"], n=20)
top_neg = top_tfidf_words(df_tfidf.loc["neg.txt"], n=20)

print("Top positive words:")
for word, score in top_pos:
    print(word,":", score)
    
print("Top negative words:")
for word, score in top_neg:
    print(word,":", score)
    
def plot_top_words(top_list, title, ax):
    words = [w for w, _ in top_list][::-1]
    scores = [s for _, s in top_list][::-1]
    y_pos = np.arange(len(words))
    ax.barh(y_pos, scores, color="skyblue", edgecolor="k")
    ax.set_yticks(y_pos)
    ax.set_xlabel("TF-IDF score")
    ax.set_title(title)
    ax.invert_yaxis()
    
fig, axes = plt.subplots(1,2)
plot_top_words(top_pos, "Top TF-IDF words - pos.txt", axes[0])
plot_top_words(top_neg, "Top TF-IDF words - neg.txt", axes[1])
plt.tight_layout()
plt.show()


**Remember to submit your code on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**

## Exercise 4 | Junk charts

There’s a thriving community of chart enthusiasts who keep looking for statistical graphics that they find inappropriate, and which they call “junk charts”, and who often also propose ways to improve them.

1. Find at least three statistical visualizations you think are not very good and identify their problems. Copying examples from various junk chart websites is not accepted – you should find your own junk charts, out in the wild. You should be able to find good (or rather, bad) examples quite easily since a significant fraction of charts can have at least *some* issues. The examples you choose should also have different problems, e.g., try to avoid collecting three bar charts, all with problematic axes. Instead, try to find as interesting and diverse examples as you can.

2. Try to produce improved versions of the charts you selected. The data is of course often not available, but perhaps you can try to extract it, at least approximately, from the chart. Or perhaps you can simulate data that looks similar enough to make the point.



**Submit a PDF with all the charts (the ones you found and the ones you produced).**